# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [13]:
import pandas as pd
import numpy as np
recipes = pd.read_csv("recipes.tsv", sep="\t")
recipes.head()
# Define preference keywords
likes = ["chicken", "beef", "pork", "bacon", "cheese", "steak", "spicy", "grill"]
dislikes = ["cake", "cookie", "pie", "dessert", "sweet", "chocolate", "frosting"]

def assign_rating(slug):
    slug_lower = slug.lower()

    # Strong dislike → 0
    if any(word in slug_lower for word in dislikes):
        return 0.0

    # Strong like → 1
    if any(word in slug_lower for word in likes):
        return 1.0

    # Neutral foods → mid rating
    return 0.5

recipes["rating"] = recipes["recipe_slug"].apply(assign_rating)
 #Ensure at least 10 zeros
zero_count = (recipes["rating"] == 0).sum()
if zero_count < 10:
    extra = recipes[recipes["rating"] == 0.5].head(10 - zero_count).index
    recipes.loc[extra, "rating"] = 0.0

# Ensure at least 10 ones
one_count = (recipes["rating"] == 1).sum()
if one_count < 10:
    extra = recipes[recipes["rating"] == 0.5].tail(10 - one_count).index
    recipes.loc[extra, "rating"] = 1.0
    atings = recipes[["recipe_slug", "rating"]].copy()

ratings = recipes[["recipe_slug", "rating"]].copy()

ratings.to_csv(
    "ratings.tsv",
    sep="\t",
    index=False
)

ratings.head()


,recipe_slug,rating
0,falafel,0.5
1,spamburger,0.5
2,bacon-fried-rice,1.0
3,chicken-fingers,1.0
4,apple-crisp,0.5


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [17]:


# Load ratings (this defines row order)
ratings = pd.read_csv("ratings.tsv", sep="\t")
ratings_slugs = ratings["recipe_slug"].tolist()

# Load recipe tags
recipe_tags = pd.read_csv("recipe-tags.tsv", sep="\t")

# Keep only rated recipes
recipe_tags = recipe_tags[recipe_tags["recipe_slug"].isin(ratings_slugs)].copy()

# Build 0/1 tag matrix
recipe_tags["value"] = 1

tag_matrix = recipe_tags.pivot_table(
    index="recipe_slug",
    columns="recipe_tag",   # ← fixed here
    values="value",
    aggfunc="max",
    fill_value=0
)

# Reindex to match ratings.tsv order
tag_matrix = tag_matrix.reindex(ratings_slugs, fill_value=0)

# Add bias column
features = tag_matrix.reset_index()
features.insert(1, "bias", 1)

# Save features.tsv
features.to_csv("features.tsv", sep="\t", index=False)

# Sanity check
print("Same order as ratings:", features["recipe_slug"].tolist() == ratings_slugs)
print("Shape:", features.shape)
features.head()


Same order as ratings: True
Shape: (100, 298)


recipe_tag,recipe_slug,bias,alfredo,almond,american,appetizer,appetizers,apple,asiancuisine,asparagus,...,udonnoodles,vanilla,vanillaicecream,vegan,vegetables,vegetarian,warm,whippedcream,winter,yeastdough
0,falafel,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,spamburger,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bacon-fried-rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,chicken-fingers,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,apple-crisp,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [19]:
# YOUR CHANGES HERE

import pandas as pd
from sklearn.linear_model import Ridge

# Load data
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Ensure ordering matches (important!)
assert features["recipe_slug"].tolist() == ratings["recipe_slug"].tolist()

# Prepare X and y
X = features.drop(columns=["recipe_slug"])
y = ratings["rating"]

# Fit ridge regression (alpha = 1)
model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)

# Extract coefficients
coefficients = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": model.coef_
})


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [20]:
# YOUR CHANGES HERE
# Save model.tsv
coefficients.to_csv("model.tsv", sep="\t", index=False)
coefficients.head()

...

Ellipsis

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [21]:
# YOUR CHANGES HERE


# Load features
features = pd.read_csv("features.tsv", sep="\t")

# Load model coefficients
model_df = pd.read_csv("model.tsv", sep="\t")

# Separate recipe slugs
recipe_slugs = features["recipe_slug"]

# Prepare feature matrix X (must match model order)
X = features.drop(columns=["recipe_slug"])

# Align coefficient order with X columns
beta = model_df.set_index("recipe_tag").loc[X.columns]["coefficient"].values

# Compute score estimates
score_estimates = X.values @ beta

# Build output DataFrame
estimates_df = pd.DataFrame({
    "recipe_slug": recipe_slugs,
    "score_estimate": score_estimates
})

# Save file
estimates_df.to_csv("estimates.tsv", sep="\t", index=False)

estimates_df.head()


,recipe_slug,score_estimate
0,falafel,0.517439
1,spamburger,0.516563
2,bacon-fried-rice,0.983095
3,chicken-fingers,0.951014
4,apple-crisp,0.520791


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [22]:
# YOUR CHANGES HERE


# Parameters
lam = 1.0      # ridge regularization
alpha = 2.0    # LinUCB alpha

# Load aligned data
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Ensure row order matches (important)
assert features["recipe_slug"].tolist() == ratings["recipe_slug"].tolist()

recipe_slugs = features["recipe_slug"].copy()

X = features.drop(columns=["recipe_slug"]).values  # includes bias + tag columns
y = ratings["rating"].values

d = X.shape[1]

# A = λI + X^T X
A = lam * np.eye(d) + X.T @ X
A_inv = np.linalg.inv(A)

# theta_hat = A^{-1} X^T y  (ridge estimate)
theta_hat = A_inv @ (X.T @ y)

# Compute bounds for each recipe
mu_hat = X @ theta_hat                                  # estimate component
var_hat = np.einsum("ij,jk,ik->i", X, A_inv, X)         # x^T A_inv x for each row
score_bound = mu_hat + alpha * np.sqrt(var_hat)

bounds_df = pd.DataFrame({
    "recipe_slug": recipe_slugs,
    "score_bound": score_bound
})

# Save
bounds_df.to_csv("bounds.tsv", sep="\t", index=False)

bounds_df.head()


,recipe_slug,score_bound
0,falafel,2.283694
1,spamburger,2.410899
2,bacon-fried-rice,2.849251
3,chicken-fingers,2.745130
4,apple-crisp,2.297132


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [23]:
# YOUR CHANGES HERE

# Parameters (same as Part 5)
lam = 1.0
alpha = 2.0
T = 100

# Load data
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

# Create reward lookup (rating is the reward)
reward_map = dict(zip(ratings["recipe_slug"], ratings["rating"]))

# Feature matrix and recipe ids
slugs = features["recipe_slug"].tolist()
X_df = features.drop(columns=["recipe_slug"])
X = X_df.values
d = X.shape[1]

# LinUCB state: start with no data
A = lam * np.eye(d)
b = np.zeros(d)

records = []

for t in range(T):
    A_inv = np.linalg.inv(A)
    theta_hat = A_inv @ b

    # Compute LinUCB bounds for all recipes
    mu_hat = X @ theta_hat
    var_hat = np.einsum("ij,jk,ik->i", X, A_inv, X)  # x^T A_inv x for each recipe
    bounds = mu_hat + alpha * np.sqrt(var_hat)

    # Pick best recipe (ties broken by lowest index)
    chosen_idx = int(np.argmax(bounds))
    chosen_slug = slugs[chosen_idx]
    chosen_x = X[chosen_idx]

    # Observe reward from ratings.tsv
    r = float(reward_map[chosen_slug])

    # Record recommendation
    records.append({
        "recipe_slug": chosen_slug,
        "score_bound": float(bounds[chosen_idx]),
        "reward": r
    })

    # Update LinUCB (A += x x^T, b += r x)
    A += np.outer(chosen_x, chosen_x)
    b += r * chosen_x

# Save recommendations
recommendations_df = pd.DataFrame(records)
recommendations_df.to_csv("recommendations.tsv", sep="\t", index=False)
recommendations_df.head()


,recipe_slug,score_bound,reward
0,apple-crumble,7.483315,0.5
1,ma-la-chicken,7.225922,1.0
2,quesadillas,7.285068,0.5
3,ramen,7.269637,0.5
4,bacon-fried-rice,6.984416,1.0


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.